In [1]:
!pip install datasets peft accelerate
# Install kagglehub if not already installed
!pip install kagglehub pandas



ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [3]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 55.7 MB/s eta 0:00:00


In [4]:
import faiss

In [5]:
pip install -U transformers


In [6]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [7]:
from transformers import AutoTokenizer, AutoModelWithLMHead
import kagglehub
import pandas as pd
import os

In [8]:




# Step 1: Download the dataset from Kaggle
dataset_path = kagglehub.dataset_download("pythonafroz/medquad-medical-question-answer-for-ai-research")

# Step 2: Get the CSV file
csv_file = os.path.join(dataset_path, os.listdir(dataset_path)[0])

# Step 3: Load the CSV into a DataFrame
df = pd.read_csv(csv_file)
print("First 5 rows:")


dataset = df

df.head()

Using Colab cache for faster access to the 'medquad-medical-question-answer-for-ai-research' dataset.
First 5 rows:


,question,answer,source,focus_area
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea...",NIHSeniorHealth,Glaucoma
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...,NIHSeniorHealth,Glaucoma
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ...",NIHSeniorHealth,Glaucoma
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma


In [9]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
print(f"Train size: {len(train_df)}, Test size: {len(test_df)}")


Train size: 14770, Test size: 1642


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

question_col = "question"
context_col = "answer"

train_df = train_df.dropna(subset=[question_col, context_col])
test_df = test_df.dropna(subset=[question_col, context_col])

train_df[question_col] = train_df[question_col].astype(str)
train_df[context_col] = train_df[context_col].astype(str)
test_df[question_col] = test_df[question_col].astype(str)
test_df[context_col] = test_df[context_col].astype(str)

train_encodings = tokenizer(
    train_df[question_col].tolist(),
    train_df[context_col].tolist(),
    truncation=True,
    padding="max_length",
    max_length=256
)

test_encodings = tokenizer(
    test_df[question_col].tolist(),
    test_df[context_col].tolist(),
    truncation=True,
    padding="max_length",
    max_length=256
)

train_encodings["start_positions"] = [0] * len(train_encodings["input_ids"])
train_encodings["end_positions"]   = [len(ids) - 1 for ids in train_encodings["input_ids"]]

test_encodings["start_positions"] = [0] * len(test_encodings["input_ids"])
test_encodings["end_positions"]   = [len(ids) - 1 for ids in test_encodings["input_ids"]]

# Custom dataset class
class QADataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return len(self.encodings['input_ids'])
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

train_dataset = QADataset(train_encodings)
test_dataset = QADataset(test_encodings)




In [ ]:
print(train_encodings)

In [ ]:
# Load base model
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

# Apply LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "key", "value"],
    lora_dropout=0.05,
    bias="none",
    task_type="QUESTION_ANS"
    )

model = get_peft_model(model, lora_config)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    do_eval=True,
    evaluate_during_training=True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    num_train_epochs=3,
    logging_dir="./logs",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
trainer.train()

In [ ]:
eval_results = trainer.evaluate(eval_dataset=test_dataset)

print("Validation/Test Results:")
print(eval_results)

In [ ]:
from datasets import load_dataset

# Load the PubMed dataset
pubmed_data = load_dataset("MedRAG/pubmed", split="train")
contexts = pubmed_data["text"][:10000]  # Limit to first 10,000 snippets for efficiency


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Initialize the sentence transformer model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Encode the contexts to obtain embeddings
context_embeddings = embedding_model.encode(contexts, show_progress_bar=True)

# Convert embeddings to float32
context_embeddings = np.array(context_embeddings).astype("float32")

# Initialize FAISS index
dimension = context_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
index.add(context_embeddings)


In [ ]:
def retrieve(query, k=3):
    # Encode the query to obtain its embedding
    query_embedding = embedding_model.encode([query], show_progress_bar=False)
    query_embedding = np.array(query_embedding).astype("float32")

    # Perform the search
    distances, indices = index.search(query_embedding, k)

    # Retrieve the corresponding contexts
    retrieved_contexts = [contexts[i] for i in indices[0]]
    return retrieved_contexts


In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

# Define the prompt template
prompt_template = """
Given the context below, answer the question at the end.

Context:
{context}

Question:
{question}

Answer:
"""
prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)

# Initialize the question answering pipeline
qa_pipeline = pipeline(
    "question-answering",
    model=model,  # Your fine-tuned model
    tokenizer=tokenizer
)

# Wrap the pipeline with LangChain
qa_chain = RetrievalQA.from_chain_type(
    llm=HuggingFacePipeline(pipeline=qa_pipeline),
    retriever=retrieve,
    return_source_documents=True
)


In [ ]:
# Define a sample question
sample_question = "What are the latest treatments for hypertension?"

# Get the answer
result = qa_chain.run(sample_question)

# Display the answer
print("Answer:", result["result"])

# Optionally, display the retrieved contexts
print("\nRetrieved Contexts:")
for context in result["source_documents"]:
    print("-", context)
